In [1]:
import numpy as np
import pandas as pd
import scipy.stats as scs
import altair as alt

from os import path
from IPython.display import display as dsp

In [28]:
def make_top_fig(data_path, figname, save_to, save_as=None):
    # Load data, select columns
    df = pd.read_csv(data_path)[['sid','group','trial','activity']]
    
    # Select free-play trials
    df = df.loc[(df.trial <= 60+250) & (df.trial > 60)]
    df.loc[:, 'trial'] -= 60
    
    # Get group sizes
    group_sizes = [group_sids.size for group_sids in df.groupby(['group']).sid.unique().values]
    
    # Count subjects per activity per trial for each group
    df_stats = df.groupby(['group','activity','trial']).agg({'sid': 'nunique'})
    df_stats = df_stats.rename(columns={'sid':'n'})
    
    # Count subjects per group
    df_group_counts = df.groupby('group').agg({'sid': 'nunique'})
    df_group_counts = df_group_counts.rename(columns={'sid':'group_size'})
    
    # Calculate group-level selection relative frequencies
    df_stats = df_stats.reset_index().merge(df_group_counts, on='group')
    df_stats['rf'] = df_stats.n / df_stats.group_size
    
    charts = []
    # Visualize task choices with Altair
    for group, gdf in df_stats.groupby('group'):
        chart = alt.Chart(gdf).mark_line().encode(
            alt.X('trial',
                scale=alt.Scale(domain=(1, 250))
            ),
            alt.Y('rf',
                scale=alt.Scale(domain=(0, .5))
            ),
            color='activity'
        )
        charts.append(chart)
        
        
    chart = alt.vconcat(*charts)
    chart = chart.configure_view(
        continuousHeight=200,
        continuousWidth=400,
    )
    dsp(chart)
        

make_top_fig(
    data_path = 'data/clean_data.csv',
    figname = 'activity_preferences',
    save_to = 'figures',
    save_as = ''
)

alt.VConcatChart(...)